# 📊 EDA & Data Quality Analysis for Business Intelligence Dataset

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Excel file
excel_path = "myexc1.xlsx"
orders_df = pd.read_excel(excel_path, sheet_name='Orders', parse_dates=["Order Date", "Ship Date"])
returns_df = pd.read_excel(excel_path, sheet_name='Returns')
people_df = pd.read_excel(excel_path, sheet_name='People')

# Merge returns into orders
orders_df['Returned'] = orders_df['Order ID'].isin(returns_df['Order ID']).map({True: 'Yes', False: 'No'})


## ✅ Dataset Overview

In [ ]:

print("Rows:", orders_df.shape[0])
print("Columns:", orders_df.shape[1])
orders_df.info()


## 🧼 Missing Values

In [ ]:
orders_df.isnull().sum()

## 🧾 Summary Statistics (Numerical Columns)

In [ ]:
orders_df.describe()

## 🆔 Duplicate Rows

In [ ]:
orders_df.duplicated().sum()

## ⚠️ Invalid or Unexpected Values

In [ ]:

# Negative profit or sales
negative_profit = (orders_df['Profit'] < 0).sum()
negative_sales = (orders_df['Sales'] <= 0).sum()

# Ship date before order date
invalid_ship_dates = (orders_df['Ship Date'] < orders_df['Order Date']).sum()

# Quantity check
zero_quantity = (orders_df['Quantity'] <= 0).sum()

# Discount range check
invalid_discounts = orders_df[(orders_df['Discount'] < 0) | (orders_df['Discount'] > 0.8)].shape[0]

{
    "Negative Profit Rows": negative_profit,
    "Negative or Zero Sales": negative_sales,
    "Ship Dates Before Order Dates": invalid_ship_dates,
    "Zero or Negative Quantity": zero_quantity,
    "Invalid Discount Values": invalid_discounts
}


## 🧮 Unique Values in Categorical Columns

In [ ]:
orders_df.select_dtypes(include='object').nunique().sort_values(ascending=False)

## 📊 Visual Analysis

In [ ]:

# Create delivery lag field
orders_df['Delivery Lag (Days)'] = (orders_df['Ship Date'] - orders_df['Order Date']).dt.days

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
orders_df = pd.read_csv("merged_orders.csv", parse_dates=["Order Date", "Ship Date"])

# Create delivery lag field
orders_df['Delivery Lag (Days)'] = (orders_df['Ship Date'] - orders_df['Order Date']).dt.days

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


In [ ]:

# Plot 1: Monthly Sales Trend
orders_df.set_index('Order Date').resample('M')['Sales'].sum().plot(title="Monthly Sales Trend")
plt.ylabel("Sales ($)")
plt.xlabel("Order Date")
plt.tight_layout()
plt.savefig("monthly_sales_trend.png")
plt.close()


In [ ]:

# Plot 2: Profit Distribution
sns.histplot(orders_df['Profit'], bins=50, kde=True)
plt.title("Profit Distribution")
plt.xlabel("Profit ($)")
plt.ylabel("Frequency")
plt.tight_layout()
plt.savefig("profit_distribution.png")
plt.close()


In [ ]:

# Plot 3: Profit by Product Category
sns.boxplot(x='Category', y='Profit', data=orders_df)
plt.title("Profit by Product Category")
plt.tight_layout()
plt.savefig("profit_by_category.png")
plt.close()


In [ ]:

# Plot 4: Return Rate by Sub-Category
returns_rate = orders_df.groupby('Sub-Category')['Returned'].apply(lambda x: (x == 'Yes').mean()).sort_values(ascending=False)
returns_rate.plot(kind='bar', color='tomato')
plt.title("Return Rate by Sub-Category")
plt.ylabel("Return Rate (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("return_rate_by_subcategory.png")
plt.close()


In [ ]:

# Plot 5: Delivery Lag by Ship Mode
sns.boxplot(x='Ship Mode', y='Delivery Lag (Days)', data=orders_df)
plt.title("Shipping Delay by Ship Mode")
plt.tight_layout()
plt.savefig("delivery_lag_by_ship_mode.png")
plt.close()


In [ ]:

# Plot 6: Sales and Profit by Region
region_group = orders_df.groupby('Region')[['Sales', 'Profit']].sum().sort_values(by='Sales', ascending=False)
region_group.plot(kind='bar')
plt.title("Sales and Profit by Region")
plt.ylabel("Amount ($)")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig("sales_profit_by_region.png")
plt.close()


In [ ]:

# Plot 7: Customer Segmentation
customer_summary = orders_df.groupby('Customer ID').agg({
    'Sales': 'sum',
    'Order ID': 'nunique',
    'Profit': 'sum'
}).rename(columns={'Order ID': 'Num Orders'})
customer_summary['AOV'] = customer_summary['Sales'] / customer_summary['Num Orders']

sns.scatterplot(data=customer_summary, x='Num Orders', y='AOV', size='Profit', hue='Profit', palette='viridis', sizes=(20, 200))
plt.title("Customer Segmentation: Orders vs. AOV")
plt.xlabel("Number of Orders")
plt.ylabel("Average Order Value ($)")
plt.tight_layout()
plt.savefig("customer_segmentation.png")
plt.close()
